In [209]:
# Enrique Ulises Báez Gómez Tagle && Mauricio Iván Ascencio Martínez
# Importar nltk y tokenizadores necesarios para el procesamiento de texto.
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize.treebank import TreebankWordDetokenizer
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('spanish'))

from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.utils.np_utils import to_categorical

import matplotlib.pyplot as plt
import gradio as gr

import random

from numpy import asarray
from numpy import zeros

import re
import json
import locale
import platform
from datetime import datetime
import numpy

if platform.system() == 'Linux':
    locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')
elif platform.system() == 'Windows':
    locale.setlocale(locale.LC_TIME, 'Spanish_Spain.1252')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/enriquegomeztagle/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [210]:
# Leer y asignar datos desde archivos JSON para entrenamiento de modelos.

with open('Intenciones_NivelI.json', encoding='utf-8') as file:
    data_NivelI = json.load(file)

with open('Intenciones_NivelIIA.json', encoding='utf-8') as file:
    data_NivelIIA = json.load(file)

with open('Intenciones_NivelIIB.json', encoding='utf-8') as file:
    data_NivelIIB = json.load(file)


In [211]:
# Crear diccionarios para almacenar intenciones en distintos niveles.
NI = dict()
NIIA = dict()
NIIB = dict()

for info in data_NivelI['intents']:
    NI.setdefault(info['tag'], info['patterns'])

for info in data_NivelIIA['intents']:
    NIIA.setdefault(info['tag'], info['patterns'])

for info in data_NivelIIB['intents']:
    NIIB.setdefault(info['tag'], info['patterns'])

# print(NI)
# print(NIIA)
# print(NIIB)


In [212]:
# Generar listas de etiquetas para clasificar las intenciones en los distintos niveles.
Y_NI = list()
Y_NIIA = list()
Y_NIIB = list()

for clase, lista_textos in NI.items():
    for text in lista_textos:
        Y_NI.append(list(NI.keys()).index(clase))

for clase, lista_textos in NIIA.items():
    for text in lista_textos:
        Y_NIIA.append(list(NIIA.keys()).index(clase))

for clase, lista_textos in NIIB.items():
    for text in lista_textos:
        Y_NIIB.append(list(NIIB.keys()).index(clase))

# print("Vector de salidas Y para N1:")
# print(Y_NI)
# print("Vector de salidas Y para N2A:")
# print(Y_NIIA)
# print("Vector de salidas Y para N2B:")
# print(Y_NIIB)


In [213]:
# Definir función para eliminar stopwords de los textos.
def quitar_stopwords(Textos):
    X = list()
    for sentence in Textos:
        for stopword in stop_words:
            sentence = sentence.replace(" " + stopword + " ", " ")
        sentence = sentence.replace("á", "a")
        sentence = sentence.replace("é", "e")
        sentence = sentence.replace("í", "i")
        sentence = sentence.replace("ó", "o")
        sentence = sentence.replace("ú", "u")

        sentence = re.sub(r'\s+', ' ', sentence)
        sentence = sentence.lower()
        tokenizer = RegexpTokenizer(r'\w+')
        result = tokenizer.tokenize(sentence)
        X.append(TreebankWordDetokenizer().detokenize(result))
    return X


In [214]:
# Recolectar textos sin stopwords para nivel I.
Textos_NI = list()
for Lista in NI.values():
    for Texto in Lista:
        Textos_NI.append(Texto)

X_NI = quitar_stopwords(Textos_NI)

Textos_NIIA = list()
for Lista in NIIA.values():
    for Texto in Lista:
        Textos_NIIA.append(Texto)

X_NIIA = quitar_stopwords(Textos_NIIA)

Textos_NIIB = list()
for Lista in NIIB.values():
    for Texto in Lista:
        Textos_NIIB.append(Texto)

X_NIIB = quitar_stopwords(Textos_NIIB)


In [215]:

# print(X_NI)
# print(X_NIIA)
# print(X_NIIB)


In [216]:
# Definir tokenizadores y preparar datos para entrenamiento de modelos.
maxlen = 5

tokenizer_NI = Tokenizer(num_words=5000)
tokenizer_NIIA = Tokenizer(num_words=5000)
tokenizer_NIIB = Tokenizer(num_words=5000)

tokenizer_NI.fit_on_texts(X_NI)
X_NI_Tok = tokenizer_NI.texts_to_sequences(X_NI)
tokenizer_NIIA.fit_on_texts(X_NIIA)
X_NIIA_Tok = tokenizer_NIIA.texts_to_sequences(X_NIIA)
tokenizer_NIIB.fit_on_texts(X_NIIB)
X_NIIB_Tok = tokenizer_NIIB.texts_to_sequences(X_NIIB)


X_NI_train = pad_sequences(X_NI_Tok, padding='post', maxlen=maxlen)
X_NIIA_train = pad_sequences(X_NIIA_Tok, padding='post', maxlen=maxlen)
X_NIIB_train = pad_sequences(X_NIIB_Tok, padding='post', maxlen=maxlen)


In [217]:

# print("Matriz de entrada para NI:")
# print(X_NI_train)


In [218]:

# print("Matriz de entrada para NIIA:")
# print(X_NIIA_train)


In [219]:

# print("Matriz de entrada para NIIB:")
# print(X_NIIB_train)


In [220]:
# Cargar y procesar el archivo de embeddings de palabras para el español.
embeddings_dictionary = dict()
Embeddings_file = open('Word2Vect_Spanish.txt', encoding="utf8")

for linea in Embeddings_file:
    caracts = linea.split()
    palabra = caracts[0]
    vector = asarray(caracts[1:], dtype='float32')
    embeddings_dictionary[palabra] = vector
Embeddings_file.close()


In [221]:
# Definir función para asignar embeddings a palabras según el tokenizador.
def Asignar_Embeddings(tokenizer, vocab_size):
    embedding_matrix = zeros((vocab_size, 300))
    for word, index in tokenizer.word_index.items():
        embedding_vector = embeddings_dictionary.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector
    return embedding_matrix


In [222]:
# Asignar matrices de embeddings a los diferentes niveles y preparar tokenizadores.
vocab_size_NI = len(tokenizer_NI.word_index) + 1
embedding_matrix_NI = Asignar_Embeddings(tokenizer_NI, vocab_size_NI)

vocab_size_NIIA = len(tokenizer_NIIA.word_index) + 1
embedding_matrix_NIIA = Asignar_Embeddings(tokenizer_NIIA, vocab_size_NIIA)

vocab_size_NIIB = len(tokenizer_NIIB.word_index) + 1
embedding_matrix_NIIB = Asignar_Embeddings(tokenizer_NIIB, vocab_size_NIIB)


In [223]:
# Definir modelos de aprendizaje profundo para cada nivel de intención.
def Definir_Modelos(vocab_size, embedding_matrix, X_train, labels):
    model = Sequential()
    embedding_layer = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=X_train.shape[1],
                                trainable=False)
    model.add(embedding_layer)
    model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(len(labels), activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # print(model.summary())

    # print("\nPalabras en el vocabulario:")
    # print(vocab_size)
    return model


In [224]:

model_NI = Definir_Modelos(vocab_size_NI, embedding_matrix_NI, X_NI_train, NI.keys())


In [225]:

model_NIIA = Definir_Modelos(vocab_size_NIIA, embedding_matrix_NIIA, X_NIIA_train, NIIA.keys())


In [226]:

model_NIIB = Definir_Modelos(vocab_size_NIIB, embedding_matrix_NIIB, X_NIIB_train, NIIB.keys())


In [227]:
# Entrenamiento de modelos
def Entrenar_Modelos(X_train, Y, model, labels):
    train_labels = to_categorical(Y, num_classes=len(labels))
    # print('Matriz de salidas')
    # print(train_labels)

    history = model.fit(X_train, train_labels, epochs=30, batch_size=1, verbose=1)

    score = model.evaluate(X_train, train_labels, verbose=1)
    # print("\nTest Loss:", score[0])
    # print("Test Accuracy:", score[1])
    return history


In [228]:

history_NI = Entrenar_Modelos(X_NI_train, Y_NI, model_NI, NI.keys())


Epoch 1/30
71/71 [==============================] - 1s 4ms/step - loss: 1.5160 - accuracy: 0.2958
Epoch 2/30
71/71 [==============================] - 0s 5ms/step - loss: 0.9671 - accuracy: 0.7042
Epoch 3/30
71/71 [==============================] - 0s 5ms/step - loss: 0.4568 - accuracy: 0.8592
Epoch 4/30
71/71 [==============================] - 0s 5ms/step - loss: 0.2220 - accuracy: 0.9437
Epoch 5/30
71/71 [==============================] - 0s 5ms/step - loss: 0.2630 - accuracy: 0.9014
Epoch 6/30
71/71 [==============================] - 0s 5ms/step - loss: 0.1273 - accuracy: 0.9577
Epoch 7/30
71/71 [==============================] - 0s 5ms/step - loss: 0.0363 - accuracy: 1.0000
Epoch 8/30
71/71 [==============================] - 0s 5ms/step - loss: 0.0990 - accuracy: 0.9577
Epoch 9/30
71/71 [==============================] - 0s 5ms/step - loss: 0.0791 - accuracy: 0.9859
Epoch 10/30
71/71 [==============================] - 0s 5ms/step - loss: 0.0167 - accuracy: 1.0000
Epoch 11/30
71/71 [

In [229]:

history_NIIA = Entrenar_Modelos(X_NIIA_train, Y_NIIA, model_NIIA, NIIA.keys())


Epoch 1/30
55/55 [==============================] - 1s 4ms/step - loss: 1.3268 - accuracy: 0.4182
Epoch 2/30
55/55 [==============================] - 0s 4ms/step - loss: 0.8229 - accuracy: 0.8727
Epoch 3/30
55/55 [==============================] - 0s 5ms/step - loss: 0.3050 - accuracy: 0.9091
Epoch 4/30
55/55 [==============================] - 0s 5ms/step - loss: 0.2593 - accuracy: 0.9273
Epoch 5/30
55/55 [==============================] - 0s 5ms/step - loss: 0.0417 - accuracy: 1.0000
Epoch 6/30
55/55 [==============================] - 0s 5ms/step - loss: 0.0142 - accuracy: 1.0000
Epoch 7/30
55/55 [==============================] - 0s 5ms/step - loss: 0.0065 - accuracy: 1.0000
Epoch 8/30
55/55 [==============================] - 0s 5ms/step - loss: 0.0030 - accuracy: 1.0000
Epoch 9/30
55/55 [==============================] - 0s 5ms/step - loss: 0.0025 - accuracy: 1.0000
Epoch 10/30
55/55 [==============================] - 0s 5ms/step - loss: 0.0018 - accuracy: 1.0000
Epoch 11/30
55/55 [

In [230]:

history_NIIB = Entrenar_Modelos(X_NIIB_train, Y_NIIB, model_NIIB, NIIB.keys())


Epoch 1/30
22/22 [==============================] - 1s 3ms/step - loss: 1.0760 - accuracy: 0.4091
Epoch 2/30
22/22 [==============================] - 0s 4ms/step - loss: 0.9311 - accuracy: 0.4545
Epoch 3/30
22/22 [==============================] - 0s 4ms/step - loss: 0.7362 - accuracy: 0.5455
Epoch 4/30
22/22 [==============================] - 0s 4ms/step - loss: 0.4804 - accuracy: 0.9545
Epoch 5/30
22/22 [==============================] - 0s 5ms/step - loss: 0.1721 - accuracy: 1.0000
Epoch 6/30
22/22 [==============================] - 0s 6ms/step - loss: 0.0373 - accuracy: 1.0000
Epoch 7/30
22/22 [==============================] - 0s 5ms/step - loss: 0.0143 - accuracy: 1.0000
Epoch 8/30
22/22 [==============================] - 0s 5ms/step - loss: 0.0035 - accuracy: 1.0000
Epoch 9/30
22/22 [==============================] - 0s 5ms/step - loss: 0.0018 - accuracy: 1.0000
Epoch 10/30
22/22 [==============================] - 0s 5ms/step - loss: 0.0017 - accuracy: 1.0000
Epoch 11/30
22/22 [

In [231]:
# Función para evaluación de rendimiento
def Grafica_Modelo(history):
    plt.figure(figsize=(12, 5))
    plt.ylim(-0.1, 1.1)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['loss'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['Acc', 'Loss'])
    plt.show()


In [232]:

# Grafica_Modelo(history_NI)


In [233]:

# Grafica_Modelo(history_NIIA)


In [234]:

# Grafica_Modelo(history_NIIB)


# Programación del Árbol conversacional

In [235]:
# Instancer base para poder obtener un tag y su score maximo
def instancer(inp, model, tags):
    inp = inp.lower().replace("á", "a").replace("é", "e").replace("í", "i").replace("ó", "o")
    inp = inp.replace("ú", "u").replace("¿", "").replace("?", "")
    txt = [inp]
    seq = tokenizer_NI.texts_to_sequences(txt)
    padded = pad_sequences(seq, maxlen=maxlen)

    # print("Input shape before predict:", padded.shape)
    # print("Input data:", padded)

    results = model.predict(padded)
    results_index = numpy.argmax(results)
    tag = list(tags.keys())[results_index]
    maxscore = numpy.max(results)
    return tag, maxscore


In [236]:

saludos_usuario = [
    'hola', 'buenas noches', 'buenas tardes', 'buenos días', 'qué tal',
    'qué onda', 'cómo estás', 'cómo va', 'cómo te va', 'qué pasa', 
    'qué más', 'cómo andas', 'cómo va todo', 'qué hay', 'cómo estáis',
    'cómo están', 'qué hay de nuevo', 'qué se cuenta', 'cómo te ha ido',
    'qué cuentas', 'cómo lo llevas', 'qué tal todo', 'qué tal estás',
    'saludos', 'buen día', 'ey', 'ola', 'buenas', 'Hola'
]

saludos_chatbot = ['¡Hola! ¿Cómo puedo ayudarte?', 'Qué tal ¿en qué puedo asistirte?', 'Buenas, ¿cómo puedo servirte?', 'Hola de nuevo, ¿qué necesitas?', '¡Hola!']

despedidas_usuario = [
    'adiós', 'hasta luego', 'nos vemos', 'hasta pronto', 'hasta la vista',
    'chao', 'bye', 'hasta mañana', 'hasta la próxima', 'me voy', 'me tengo que ir',
    'tengo que irme', 'cuídate', 'te veo después', 'nos hablamos', 'hasta más tarde',
    'hasta entonces', 'hasta después', 'que estés bien', 'que te vaya bien',
    'cuidate mucho', 'te dejo', 'se me hace tarde', 'tengo que salir'
]

despedidas_chatbot = [
    '¡Adiós! Espero haber sido de ayuda.', 'Hasta luego. Si necesitas algo más, aquí estaré.',
    '¡Nos vemos! Que tengas un buen día.', 'Hasta pronto, ¡fue un placer asistirte!',
    'Que te vaya bien. ¡Gracias por visitarnos!'
]

agradecimientos_usuario = [
    'gracias', 'te agradezco', 'muchas gracias', 'agradecido', 'se agradece',
    'aprecio tu ayuda', 'eres un sol', 'mil gracias', 'gracias por tu ayuda',
    'gracias por todo', 'te lo agradezco mucho', 'te estoy muy agradecido',
    'no sabes cuánto te lo agradezco', 'estoy agradecido', 'qué amable', 'eres muy amable'
]

agradecimientos_chatbot = [
    '¡De nada! Siempre estoy aquí para ayudarte.', 'Fue un placer asistirte, ¡no hay de qué!',
    'Me alegro de haber sido de ayuda, ¡gracias a ti!', 'Estoy aquí para lo que necesites, ¡gracias por tu amabilidad!',
    '¡Gracias a ti por usar nuestro servicio!'
]





# Nivel 1
def Activar_NI(user_input):
    if user_input is None:
        response = ""
        return response, 'NI'

    if user_input == "AUTORES":
            response = "Easter Egg: Elaborado por Mau y Kike"
            return response, 'NI'
    
    if user_input == "MEJOR PROFE":
            response = "Easter Egg: IRVING"
            return response, 'NI'
    
    if user_input == "SALUDOS A MICH":
            response = "Easter Egg: Saludos Miich, un gusto verte por acá"
            return response, 'NI'
    
    if user_input in saludos_usuario:
            response = random.choice(saludos_chatbot)
            return response, 'NI'
    
    if user_input in agradecimientos_usuario:
            response = random.choice(agradecimientos_chatbot)
            return response, 'NI'
    
    if user_input in despedidas_usuario:
            response = random.choice(despedidas_chatbot)
            return response, 'NI'

    tag, maxscore = instancer(user_input, model_NI, NI)

    if maxscore > 0.65 or user_input == 'salir':

        if tag == 'Reservar_Habitacion':
            next_state = 'NIIA1'
            response = "Entendido, estás interesado en reservar una habitación. ¿Qué tipo de habitación deseas?"

        elif tag == 'Servicio_hotel':
            if user_input.count('Apartar_espacio') > 0:
                next_state = 'NIIB1'
            elif user_input.count('Servicio') > 0:
                next_state = 'NIIB2'
            elif user_input.count('Comida_bebida') > 0:
                next_state = 'NIIB3'
            else:
                next_state = 'NIIB'
            response = "Claro, puedo ayudarte con los servicios del hotel. (pedir comida, apartar espacio,...)"

        elif tag == 'Historia_Hotel':
            response = "Desde su apertura en 1920 por el pionero Don Eduardo Mendoza, nuestro hotel es un ícono de la arquitectura Art Deco, diseñado por Carlos Fontana.\nHa acogido a luminarias como la actriz Clara Estrella en los años 30 y fue el lugar de la histórica cumbre de paz en 1955. Cada habitación cuenta una historia; por ejemplo, la suite 204, donde el novelista Luis Montero escribió su obra maestra. Es un lugar donde la historia y la hospitalidad se encuentran en cada esquina."
            next_state = 'NI'

        elif tag == 'Atracciones_Cercanas':
            response = "En el corazón de la ciudad, nuestro hotel se encuentra a un corto trayecto del Parque Central y a unos minutos del Museo Nacional de Arte, perfecto para los entusiastas de la cultura. Para aquellos interesados en la historia, el Convento de San Francisco está a una caminata de distancia. No te pierdas el Mercado del Sol, ideal para saborear la cocina local y encontrar artesanías únicas. Además, ofrecemos excursiones al Castillo de San Lorenzo, un sitio histórico con vistas espectaculares, a solo media hora en coche."
            next_state = 'NI'

        elif tag == "Famosos_hotel":
            response = "Nuestro hotel ha sido un destino favorito para muchas personalidades destacadas a lo largo de los años. Entre los huéspedes ilustres, hemos tenido el placer de acoger a figuras como el actor ganador del Oscar Leonardo DiCaprio, la aclamada cantante Adele, y el empresario tecnológico Elon Musk. Además, leyendas del deporte como Serena Williams y estrellas de la talla de Beyoncé también han disfrutado de la exclusividad y el servicio de primera clase de nuestro establecimiento. Cada celebridad que nos visita añade un capítulo único a la rica historia de nuestro hotel, asegurando que no solo sea un lugar de lujo, sino también un espacio donde se cruzan caminos extraordinarios"
            next_state = 'NI'

        else:
            next_state = 'NI'
            response = "Puedes preguntarme sobre reservaciones, servicios del hotel, su historia o atracciones cercanas."

        return response, next_state

    else:
        response = "Lo siento, pero no entendí tu petición, ¿Podrías decirlo de otra forma?"
        return response, 'NI'


In [237]:
# Nivel 2a
def Activar_NIIA(user_input):
    if user_input is None:
        response = "Escribe 'reservar habitación' para confirmar"
        return response, 'NIIA'
        
    if user_input in despedidas_usuario:
            response = random.choice(despedidas_chatbot)
            return response, 'NI'

    tag, maxscore = instancer(user_input, model_NIIA, NIIA)

    if maxscore > 0.7 or user_input == 'salir' or user_input == 'volver':
        if user_input == 'volver':
            next_state = 'NI'
            response = "Entendido, volviendo al menú principal."
        elif user_input == 'salir':
            response = "Hasta luego, fue un gusto hablar contigo."
            return response
        else:
            next_state = 'NIIA1'        

        return response, next_state
    else:
        response = "Lo siento, pero no entendí.. Puedes internatar de nuevo"
        
        return response, 'NIIA'


In [238]:
# Nivel 2a1
def Activar_NIIA1(user_input=None):
    if user_input is None:
        response = ""
        return response, 'NIIA1'
    
    user_input = user_input.lower()

    global Tipo_Cuarto
        
    if user_input in despedidas_usuario:
        response = random.choice(despedidas_chatbot)
        return response, 'NI'

    tag, maxscore = instancer(user_input, model_NIIA, NIIA)

    if maxscore > 0.5 or user_input == 'salir' or user_input == 'volver':
        if 'estándar' in user_input or 'económica' in user_input:
            response = "Excelente, reservaremos una habitación estándar. ¿Para cuándo agendamos su entrada? (dd/mm/aaaa)"
            Tipo_Cuarto = "Estándar"
            next_state = 'NIIA2'
        elif 'suite' in user_input:
            response = "Excelente, reservaremos una habitación suite. ¿Para cuándo agendamos su entrada? (dd/mm/aaaa)"
            Tipo_Cuarto = "Suite"
            next_state = 'NIIA2'
        elif 'doble' in user_input or 'dos camas' in user_input:
            response = "Excelente, reservaremos una habitación doble. ¿Para cuándo agendamos su entrada? (dd/mm/aaaa)"
            Tipo_Cuarto = "Doble"
            next_state = 'NIIA2'
        elif 'familiar' in user_input:
            response = "Excelente, reservaremos una habitación familiar. ¿Para cuándo agendamos su entrada? (dd/mm/aaaa)"
            Tipo_Cuarto = "Familiar"
            next_state = 'NIIA2'
        elif 'deluxe' in user_input:
            response = "Excelente, reservaremos una habitación deluxe. ¿Para cuándo agendamos su entrada? (dd/mm/aaaa)"
            Tipo_Cuarto = "Deluxe"
            next_state = 'NIIA2'
        elif 'ofrecen' in user_input:
            response = "Ofrecemos los siguientes tipos de habitaciones: estándar, doble, familiar, suite, y deluxe."
        elif user_input == 'volver':
            next_state = 'NIIA'
            response = "De acuerdo, volviendo al menú anterior."
        elif user_input == 'salir':
            next_state = 'salir'
            response = "Hasta luego, fue un gusto ayudarte."
        else:
            next_state = 'NIIA1'
            response = "No he podido identificar el tipo de habitación. ¿Puedes especificar si deseas una habitación estándar, suite, doble, etc.?"

        return response, next_state


In [239]:
# nivel 2a2
def Activar_NIIA2(user_input):
    if user_input is None:
        response = ""
    global Fecha_entrada
    user_input = user_input.lower()
    
    if user_input in despedidas_usuario:
            response = random.choice(despedidas_chatbot)
            return response, 'NI'
    else:
        try:
            fecha_entrada_usuario = datetime.strptime(user_input, '%d/%m/%Y')
            if fecha_entrada_usuario.date() < datetime.now().date():
                response = "No puedes agendar una fecha en el pasado. Por favor, elige una fecha futura."
                next_state = 'NIIA2'
            else:
                fecha_formateada = fecha_entrada_usuario.strftime('%d de %B del %Y')
                response = f"Muy bien, agendaré la entrada para el {fecha_formateada}. ¿Para cuándo agendamos su salida? (dd/mm/aaaa)"
                Fecha_entrada = fecha_entrada_usuario
                next_state = 'NIIA3'
        except ValueError:
            response = "Parece que la fecha no está en el formato correcto. Por favor, ingresa la fecha en formato dd/mm/aaaa."
            next_state = 'NIIA2'

    return response, next_state


In [240]:
# nivel2a3
def Activar_NIIA3(user_input):
    if user_input is None:
        response = ""

    global Fecha_salida
    user_input = user_input.lower()

    if user_input in despedidas_usuario:
            response = random.choice(despedidas_chatbot)
            return response, 'NI'
    else:
        try:
            fecha_salida_usuario = datetime.strptime(user_input, '%d/%m/%Y')
            if fecha_salida_usuario.date() < datetime.now().date():
                response = "No puedes agendar una fecha de salida en el pasado. Por favor, elige una fecha futura."
            else:
                fecha_formateada = fecha_salida_usuario.strftime('%d de %B del %Y')
                response = f"Muy bien, agendaré la salida para el {fecha_formateada}. ¿Para cuántos huéspedes?"
                Fecha_salida = fecha_salida_usuario
                next_state = 'NIIA4'
        except ValueError:
            response = "Parece que la fecha no está en el formato correcto. Por favor, ingresa la fecha en formato dd/mm/aaaa."
            next_state = 'NIIA3'

    return response, next_state


In [241]:
# Nivel 2 a 4
def Activar_NIIA4(user_input):
    if user_input is None:
        response = ""
        return response, 'NI'

    user_input = user_input.lower()
    global Num_huespedes

    Num_huespedes = user_input

    if user_input in agradecimientos_usuario:
        response = random.choice(agradecimientos_chatbot)
        response += "\nReservación completada.. Presiona 'CLEAR' para continuar"
        return response, 'NI'
    
    if user_input in despedidas_usuario:
        response = random.choice(despedidas_chatbot)
        response += "\nReservación completada.. Presiona 'CLEAR' para continuar"
        return response, 'NI'
    
    if user_input == "RESUMEN":
        response = "El resumen de tu reservación es ", Tipo_Cuarto, " de los días ", Fecha_entrada, " a ", Fecha_salida, " para ", Num_huespedes, " personas."
        return response, 'NI'
    else:
        response = "Reservación completada.. Presiona 'CLEAR' para continuar"
        next_state = 'NI'

    return response, next_state


In [242]:
#nivel 2b
def Activar_NIIB(user_input=None):
    if user_input is None:
        response = ""

    if user_input in despedidas_usuario:
            response = random.choice(despedidas_chatbot)
            return response, 'NI'

    tag, maxscore = instancer(user_input, model_NIIB, NIIB)

    if maxscore > 0.2 or user_input == 'salir' or user_input == 'volver':
        if user_input == 'volver':
            next_state = 'NI'
            response = "De acuerdo, volviendo al menú anterior."
        elif user_input == 'salir':
            response = "Hasta luego, fue un gusto hablar contigo."
            next_state = 'salir'
        elif tag == 'Apartar_espacio':
            next_state = 'NIIB1'
            response = "Excelente! Apartemos un espacio. ¿Qué tipo de espacio te gustaría reservar?"
        elif tag == 'Servicio':
            next_state = 'NIIB2'
            response = "Con gusto! ¿Qué servicio le gustaría pedir"
        elif tag == 'Comida_bebida':
            next_state = 'NIIB3'
            response = "Hora de comer! ¿Qué alimento o bebida le gustaría pedir?"
        else:
            response = "No estoy seguro de cómo ayudarte con eso. ¿Podrías especificar más tu solicitud?"
            next_state = 'NIIB'
    else:
        response = "Lo siento, pero no entendí. ¿Podrías ser más específico?"
        next_state = 'NIIB'

    return response, next_state


In [243]:
# Nivel 2b1
def Activar_NIIB1(user_input):
    if user_input is None:
        response = "Creo que no dijiste nada, ¿Qué tipo de espacio te gustaría reservar?"
        next_state = 'NIIB1'
        return response, next_state
    global Espacio_Apartado
    user_input = user_input.lower()

    if user_input in despedidas_usuario:
            response = random.choice(despedidas_chatbot)
            return response, 'NI'
    elif 'alberca' in user_input:
        response = "Excelente, reservaremos la alberca. Si hay otro servicio que te gustaría pedir no dudes en decírmelo!"
        Espacio_Apartado = "alberca"
        next_state = 'NI'
    elif 'gimnasio' in user_input:
        response = "Excelente, reservaremos el gimnasio. Si hay otro servicio que te gustaría pedir no dudes en decírmelo!"
        Espacio_Apartado = "gimnasio"
        next_state = 'NI'
    elif 'eventos' in user_input or 'salon' in user_input:
        response = "Excelente, reservaremos el salón de eventos. Si hay otro servicio que te gustaría pedir no dudes en decírmelo!"
        Espacio_Apartado = "salón de eventos"
        next_state = 'NI'
    elif 'terraza' in user_input:
        response = "Excelente, reservaremos la terraza. Si hay otro servicio que te gustaría pedir no dudes en decírmelo!"
        Espacio_Apartado = "terraza"
        next_state = 'NI'
    elif 'spa' in user_input:
        response = "Excelente, reservaremos el spa. Si hay otro servicio que te gustaría pedir no dudes en decírmelo!"
        Espacio_Apartado = "spa"
        next_state = 'NI'
    else:
        response = "¿Podrías repetirme qué tipo de espacio te gustaría reservar?"
        next_state = 'NIIB1'

    return response, next_state


In [244]:
#Nivel 2b2
def Activar_NIIB2(user_input=None):
    if user_input is None:
        response = "Creo que no dijiste nada, ¿Qué servicio le gustaría pedir?"
        next_state = 'NIIB2'
        return response, next_state
    global Servicio_Pedido
    user_input = user_input.lower()

    if user_input in despedidas_usuario:
            response = random.choice(despedidas_chatbot)
            return response, 'NI'
    elif user_input.count('limpiar') > 0:
        response = "Claro, enviaremos a alguien para limpiar la habitación. Si hay otro servicio que te gustaría pedir no dudes en decírmelo!"
        Servicio_Pedido = "limpieza de cuarto"
        next_state = 'NI'
    elif user_input.count('toallas') > 0:
        response = "Ok, enviaremos toallas a su habitación. Si hay otro servicio que te gustaría pedir no dudes en decírmelo!"
        Servicio_Pedido = "toallas"
        next_state = 'NI'
    elif user_input.count('jabón') > 0 or user_input.count('shampoo') > 0:
        response = "Seguro! enviaremos artículos de higiene personal a su cuarto. Si hay otro servicio que te gustaría pedir no dudes en decírmelo!"
        Servicio_Pedido = "higiene personal"
        next_state = 'NI'
    elif user_input.count('equipaje') > 0 or user_input.count('maletas') > 0:
        response = "Mandaremos a alguien para apoyar con el equipaje. Si hay otro servicio que te gustaría pedir no dudes en decírmelo!"
        Servicio_Pedido = "apoyo con equipaje"
        next_state = 'NI'
    elif user_input.count('almohadas') > 0:
        response = "Con gusto enviaremos más almohadas a la habitación. Si hay otro servicio que te gustaría pedir no dudes en decírmelo!"
        Servicio_Pedido = "almohadas"
        next_state = 'NI'
    else:
        response = "¿Podrías repetirme qué servicio requieres?"
        next_state = 'NIIB2'

    return response, next_state


In [245]:
# Nivel 2b3
def Activar_NIIB3(user_input=None):
    if user_input is None:
        response = "Creo que no dijiste nada, ¿Qué alimento o bebida le gustaría pedir?"
        next_state = 'NIIB3'
        return response, next_state

    user_input = user_input.lower()

    global comida_pedida

    if user_input in despedidas_usuario:
            response = random.choice(despedidas_chatbot)
            return response, 'NI'
    elif user_input.count('desayuno') > 0:
        response = "Claro, enviaremos el desayuno a la habitación."
        comida_pedida = "desayuno"
        next_state = 'NI'
    elif user_input.count('vegana') > 0 or user_input.count('vegetariana') > 0:
        response = "Por supuesto, prepararemos una opción vegana/vegetariana para usted."
        comida_pedida = "vegana/vegetariana"
        next_state = 'NI'
    elif user_input.count('comida') > 0:
        response = "Mandaremos la comida a la habitación."
        comida_pedida = "comida"
        next_state = 'NI'
    elif user_input.count('botella') > 0 or user_input.count('vino') > 0:
        response = "Seguro! enviaremos una botella de vino."
        comida_pedida = "vino"
        next_state = 'NI'
    elif user_input.count('café') > 0:
        response = "Con gusto enviaremos café a su habitación."
        comida_pedida = "café"
        next_state = 'NI'
    elif user_input.count('romántica') > 0:
        response = "Prepararemos una cena romántica para usted."
        comida_pedida = "romántica"
        next_state = 'NI'
    elif user_input.count('cena') > 0:
        response = "Claro, mandaremos la cena a la habitación."
        comida_pedida = "cena"
        next_state = 'NI'
    else:
        response = "¿Podrías repetirme qué alimento/bebida quieres pedir?"
        next_state = 'NIIB3'

    return response, next_state


In [246]:
# Implementación de casos correspondientes para cada nivel del ChatBot
# Nivel contextual inicial por defecto
# *Variables para guardar información

global Tipo_Cuarto
global Fecha_entrada
global Fecha_salida
global Num_huespedes
global Servicio_deseado
global Espacio_Apartado
global Servicio_Pedido

Tipo_Cuarto = ""
Fecha_entrada = ""
Fecha_salida = ""
Num_huespedes = ""
Servicio_Pedido = ""
Espacio_Apartado = ""
Servicio_Pedido = ""
comida_pedida = ""

estado_actual = "NI"

maquina_estados = {'NI': Activar_NI,
                   "NIIA": Activar_NIIA,
                   "NIIA1": Activar_NIIA1,
                   "NIIA2": Activar_NIIA2,
                   "NIIA3": Activar_NIIA3,
                   "NIIA4": Activar_NIIA4,
                   "NIIB": Activar_NIIB,
                   "NIIB1": Activar_NIIB1,
                   "NIIB2": Activar_NIIB2,
                   "NIIB3": Activar_NIIB3
                   }

# Funciones de Gradio
def adapted_chat1(user_input, Nivel):
    if user_input == "":
        response, next_state = maquina_estados[Nivel](None)
    else:
        response, next_state = maquina_estados[Nivel](user_input)
    return response, next_state

def gradio_chatbot_function(user_input, state='NI'):
    if user_input == "":
        response, next_state = adapted_chat1(None, state)
    else:
        response, next_state = adapted_chat1(user_input, state)
    return response, next_state

#interfaz de gradio
iface = gr.Interface(
    fn=gradio_chatbot_function,
    inputs=[gr.Textbox(lines=2, label="Tu mensaje", placeholder="Escribe tu mensaje aquí..."), "state"],
    outputs=[gr.Textbox(label="Respuesta del Chatbot", placeholder="Puedes preguntarme sobre reservaciones, servicios del hotel, su historia o atracciones cercanas"), "state"],
    title="Hotel ESDAI Chatbot",
    description="Bienvenido a Hotel ESDAI Chatbot. ¿Qué deseas hacer? Presiona CLEAR cada vez que completes lo que deseas",
    allow_flagging='never'
)

iface.launch()


Running on local URL:  http://127.0.0.1:7867

To create a public link, set `share=True` in `launch()`.


In [247]:
# # Check the size of the tokenizer's word index
# print("Tokenizer word index size:", len(tokenizer_NI.word_index))
# 
# # After loading the model, check the input dimension of the embedding layer
# embedding_input_dim = model_NIIB.layers[0].get_config()['input_dim']
# print("Model's embedding input dimension:", embedding_input_dim)


1/1 [==============================] - 0s 20ms/step


Traceback (most recent call last):
  File "/Users/enriquegomeztagle/anaconda3/envs/NLP/lib/python3.9/site-packages/gradio/routes.py", line 534, in predict
    output = await route_utils.call_process_api(
  File "/Users/enriquegomeztagle/anaconda3/envs/NLP/lib/python3.9/site-packages/gradio/route_utils.py", line 226, in call_process_api
    output = await app.get_blocks().process_api(
  File "/Users/enriquegomeztagle/anaconda3/envs/NLP/lib/python3.9/site-packages/gradio/blocks.py", line 1550, in process_api
    result = await self.call_function(
  File "/Users/enriquegomeztagle/anaconda3/envs/NLP/lib/python3.9/site-packages/gradio/blocks.py", line 1185, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "/Users/enriquegomeztagle/anaconda3/envs/NLP/lib/python3.9/site-packages/anyio/to_thread.py", line 28, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(func, *args, cancellable=cancellable,
  File "/Users/enriquegomeztagle/anaconda3/envs/NLP/lib

1/1 [==============================] - 0s 17ms/step


2023-11-07 12:08:05.817547: I tensorflow/core/common_runtime/executor.cc:1197] [/job:localhost/replica:0/task:0/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: indices[0,3] = 96 is not in [0, 58)
	 [[{{node sequential_13/embedding_13/embedding_lookup}}]]
Traceback (most recent call last):
  File "/Users/enriquegomeztagle/anaconda3/envs/NLP/lib/python3.9/site-packages/gradio/routes.py", line 534, in predict
    output = await route_utils.call_process_api(
  File "/Users/enriquegomeztagle/anaconda3/envs/NLP/lib/python3.9/site-packages/gradio/route_utils.py", line 226, in call_process_api
    output = await app.get_blocks().process_api(
  File "/Users/enriquegomeztagle/anaconda3/envs/NLP/lib/python3.9/site-packages/gradio/blocks.py", line 1550, in process_api
    result = await self.call_function(
  File "/Users/enriquegomeztagle/anaconda3/envs/NLP/lib/python3.9/site-packages/gradio/blocks.py", line 11